In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
#import matplotlib.pyplot as plt
#import seaborn as sns
import urllib.parse
from urllib.parse import urlparse, parse_qs
import re
from collections import Counter

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()
import warnings
warnings.filterwarnings("ignore")

errors_by_rule = Counter()
correct_by_rule = Counter()

In [ ]:
def count_by_rule(rule_name, typeof):
    if typeof == "E":
        errors_by_rule[rule_name] += 1
    elif typeof == "C":
        correct_by_rule[rule_name] += 1

In [ ]:
def is_nan_string(value):
    return (
        value is None or 
        value != value or  # NaN check 
        str(value).lower() in ['nan', 'none']
    )

In [ ]:
def remove_double_quotes_from_object_columns(df):
    df_cleaned = df.copy()
    object_columns = df_cleaned.select_dtypes(include='object').columns
    for col in object_columns:
        df_cleaned[col] = df_cleaned[col].astype(str).str.replace('"', '')
    
    return df_cleaned

In [ ]:
def create_rules_dataframe():
    rules_data = [
        {
            'RULE NAME': 'Organic Email',
            'RULE': 'utm_medium = email',
            'MARKETING_CHANNEL': 'Email',
            'MARKETING_CHANNEL_ROLLUP': 'Email',
            'MARKETING_TYPE': 'Organic'
        },
        {
            'RULE NAME': 'Organic Email',
            'RULE': 'utm_medium = emaileditorial',
            'MARKETING_CHANNEL': 'Email',
            'MARKETING_CHANNEL_ROLLUP': 'Email',
            'MARKETING_TYPE': 'Organic'
        },
        {
            'RULE NAME': 'Admedia',
            'RULE': 'utm_source = admedia',
            'MARKETING_CHANNEL': 'AdMedia',
            'MARKETING_CHANNEL_ROLLUP': 'Paid Search',
            'MARKETING_TYPE': 'Paid'
        },
        {
            'RULE NAME': 'Ad.net',
            'RULE': 'utm_source = adnet',
            'MARKETING_CHANNEL': 'Ad.net',
            'MARKETING_CHANNEL_ROLLUP': 'Paid Search',
            'MARKETING_TYPE': 'Paid'
        },
        {
            'RULE NAME': 'Paid Search',
            'RULE': 'utm_medium = sem',
            'MARKETING_CHANNEL': 'Search',
            'MARKETING_CHANNEL_ROLLUP': 'Paid Search',
            'MARKETING_TYPE': 'Paid'
        },
        {
            'RULE NAME': 'Paid Search',
            'RULE': 'utm_medium = paid_search',
            'MARKETING_CHANNEL': 'Search',
            'MARKETING_CHANNEL_ROLLUP': 'Paid Search',
            'MARKETING_TYPE': 'Paid'
        },
        {
            'RULE NAME': 'Paid Display',
            'RULE': 'utm_medium = display and utm_type = paid',
            'MARKETING_CHANNEL': 'Display',
            'MARKETING_CHANNEL_ROLLUP': 'Paid Other',
            'MARKETING_TYPE': 'Paid'
        },
        {
            'RULE NAME': 'Paid Linear',
            'RULE': 'utm_medium = radio and utm_type = paid',
            'MARKETING_CHANNEL': 'Linear',
            'MARKETING_CHANNEL_ROLLUP': 'Paid Other',
            'MARKETING_TYPE': 'Paid'
        },
        {
            'RULE NAME': 'Paid Linear',
            'RULE': 'utm_medium = linear and utm_type = paid',
            'MARKETING_CHANNEL': 'Linear',
            'MARKETING_CHANNEL_ROLLUP': 'Paid Other',
            'MARKETING_TYPE': 'Paid'
        },
        {
            'RULE NAME': 'Paid Event',
            'RULE': 'utm_medium = event and utm_type = paid',
            'MARKETING_CHANNEL': 'Event',
            'MARKETING_CHANNEL_ROLLUP': 'Paid Other',
            'MARKETING_TYPE': 'Paid'
        },
        {
            'RULE NAME': 'Paid Social',
            'RULE': 'utm_medium = social and utm_type = paid',
            'MARKETING_CHANNEL': 'Social',
            'MARKETING_CHANNEL_ROLLUP': 'Paid Social',
            'MARKETING_TYPE': 'Paid'
        },
        {
            'RULE NAME': 'True Anthem Social',
            'RULE': 'utm_campaign = true_anthem',
            'MARKETING_CHANNEL': 'Social',
            'MARKETING_CHANNEL_ROLLUP': 'Organic Social',
            'MARKETING_TYPE': 'Organic'
        },
        {
            'RULE NAME': 'Legacy Paid Social',
            'RULE': 'sm_id',
            'MARKETING_CHANNEL': 'Social',
            'MARKETING_CHANNEL_ROLLUP': 'Paid Social',
            'MARKETING_TYPE': 'Paid'
        },
        {
            'RULE NAME': 'Paid Magazine',
            'RULE': 'utm_medium = magazine and utm_type = paid',
            'MARKETING_CHANNEL': 'Magazine',
            'MARKETING_CHANNEL_ROLLUP': 'Paid Other',
            'MARKETING_TYPE': 'Paid'
        },
        {
            'RULE NAME': 'Organic Display',
            'RULE': 'utm_medium = display',
            'MARKETING_CHANNEL': 'Display',
            'MARKETING_CHANNEL_ROLLUP': 'External Referral',
            'MARKETING_TYPE': 'Organic'
        },
        {
            'RULE NAME': 'Organic Owned Social',
            'RULE': 'utm_medium = social and utm_campaign ',
            'MARKETING_CHANNEL': 'Owned Social',
            'MARKETING_CHANNEL_ROLLUP': 'Organic Social',
            'MARKETING_TYPE': 'Organic'
        },
        {
            'RULE NAME': 'Organic Social',
            'RULE': 'referrer_domain contains facebook.com instagram.com twitter.com linkedin.com pinterest.com snapchat.com tiktok.com reddit.com youtube.com tumblr.com',
            'MARKETING_CHANNEL': 'Social',
            'MARKETING_CHANNEL_ROLLUP': 'Organic Social',
            'MARKETING_TYPE': 'Organic'
        },        
        {
            'RULE NAME': 'Organic Search',
            'RULE': 'referrer_domain contains google.com bing.com yahoo.com duckduckgo.com ask.com aol.com yandex.com qwant.com startpage.com dogpile.com',
            'MARKETING_CHANNEL': 'Search',
            'MARKETING_CHANNEL_ROLLUP': 'Natural Search',
            'MARKETING_TYPE': 'Organic'
        },        
        {
            'RULE NAME': 'Organic Discover',
            'RULE': 'referrer_domain contains quicksearchbox.com googlequicksearchbox googleapis.com',
            'MARKETING_CHANNEL': 'Search',
            'MARKETING_CHANNEL_ROLLUP': 'Natural Search',
            'MARKETING_TYPE': 'Organic'
        },        
        {
            'RULE NAME': 'Web Push',
            'RULE': 'utm_medium = webpush',
            'MARKETING_CHANNEL': 'Web Push',
            'MARKETING_CHANNEL_ROLLUP': 'Web Push',
            'MARKETING_TYPE': 'Organic'
        },        
        {
            'RULE NAME': 'Web Push',
            'RULE': 'utm_campaign = webpush',
            'MARKETING_CHANNEL': 'Web Push',
            'MARKETING_CHANNEL_ROLLUP': 'Web Push',
            'MARKETING_TYPE': 'Organic'
        },        
        {
            'RULE NAME': 'Organic WBD',
            'RULE': 'referrer_domain contains warnerbros.com discovery.com hbomax.com hbo.com cnn.com cartoonnetwork.com tntdrama.com tbs.com dccomics.com hgtv.com foodnetwork.com animalplanet.com tlc.com travelchannel.com investigationdiscovery.com dcuniverse.com',
            'MARKETING_CHANNEL': 'WBD',
            'MARKETING_CHANNEL_ROLLUP': 'External Referral',
            'MARKETING_TYPE': 'Organic'
        },        
        {
            'RULE NAME': 'Organic Referring Domain',
            'RULE': 'referrer_domain exists and not contains motortrend.com hotrod.com vercel.app vercel.com localhost',
            'MARKETING_CHANNEL': 'Referring Domains',
            'MARKETING_CHANNEL_ROLLUP': 'External Referral',
            'MARKETING_TYPE': 'Organic'
        },        
        {
            'RULE NAME': 'Internal Referral',
            'RULE': 'referrer_domain contains motortrend.com hotrod.com vercel.app vercel.com',
            'MARKETING_CHANNEL': 'Internal Referral',
            'MARKETING_CHANNEL_ROLLUP': 'Direct',
            'MARKETING_TYPE': 'Organic'
        },        
        {
            'RULE NAME': 'Organic Direct',
            'RULE': 'referrer_domain not exists',
            'MARKETING_CHANNEL': 'Direct',
            'MARKETING_CHANNEL_ROLLUP': 'Direct',
            'MARKETING_TYPE': 'Organic'
        },        

    ]
    return pd.DataFrame(rules_data)

In [ ]:
def extract_utm_params(url, domain):
    url = url.lower()
    parsed_url = urlparse(url)
    query_params = parse_qs(parsed_url.query)
    utm_params = {
        'utm_medium': '',
        'utm_source': '',
        'utm_type': '',
        'utm_campaign': '',
        'sm_id':'',
        'referrer_domain':''
    }
    for key in utm_params:
        if key in query_params:
            utm_params[key] = query_params[key][0]

    if not is_nan_string(domain):
        domain = urllib.parse.urlsplit(domain)
        utm_params['referrer_domain'] = domain.netloc

    return utm_params

In [ ]:
def match_utm_rules(utm_dict, rules_df):
    # Convert input dictionary values to lowercase for case-insensitive matching
    utm_dict_lower = {k.lower(): str(v).lower() for k, v in utm_dict.items()}
    
    # Iterate through rules in order
    for _, rule in rules_df.iterrows():
        rule_str = rule['RULE'].lower()
        # Break down complex rule matching
        rule_matched = False
        
        # Handle 'contains' rules for referrer domain
        if 'referrer_domain contains' in rule_str:
            #print("1.-", rule['RULE NAME'])
            #print("2.-", rule_str)
            # Extract domains to check
            domains = rule_str.split('contains')[1].strip().split()
            #print(domains)
            # Check if referrer domain is present and contains any of the specified domains
            if 'referrer_domain' in utm_dict_lower:
                referrer_domain = utm_dict_lower.get('referrer_domain', '')
                #print("-->", referrer_domain)
                if referrer_domain != "":
                    rule_matched = any(domain in referrer_domain for domain in domains)
                #else:
                #    print("1/2 referrer_domain EMPTY!")
        elif 'referrer_domain not exists' in rule_str:
            #print("3.-", rule['RULE NAME'])
            #print("4.-", rule_str)
            if 'referrer_domain' in utm_dict_lower:
                referrer_domain = utm_dict_lower.get('referrer_domain','')
                if referrer_domain == "":
                    rule_matched = True
        elif 'referrer_domain exists and not contains' in rule_str:
            #print("5.-", rule['RULE NAME'])
            #print("6.-", rule_str)
            # Extract domains to check
            domains = rule_str.split('contains')[1].strip().split()
            #print(domains)
            # Check if referrer domain is present and contains any of the specified domains
            if 'referrer_domain' in utm_dict_lower:
                referrer_domain = utm_dict_lower.get('referrer_domain', '')
                if referrer_domain != "":
                    #print("-->", referrer_domain)
                    rule_matched = any(domain in referrer_domain for domain in domains)
                    if rule_matched == True:
                        rule_matched = False
                    else:
                        rule_matched = True
        
        # Handle simple equality checks
        else:
            # Split rule into individual conditions
            conditions = rule_str.split(' and ')
            
            # Track if all conditions are met
            condition_results = []
            
            for condition in conditions:
                # Remove any leading/trailing whitespace
                condition = condition.strip()
                
                # Check for equality conditions
                if '=' in condition:
                    key, value = [part.strip() for part in condition.split('=')]
                    
                    # Special handling for empty or non-empty checks
                    if value == '':
                        # Check if key exists but is empty
                        condition_results.append(
                            utm_dict_lower.get(key, '') == ''
                        )
                    else:
                        # Check if key exists and matches the value
                        condition_results.append(
                            utm_dict_lower.get(key, '') == value
                        )
                
                # Check for 'contains' or other conditions
                elif ' ' not in condition:
                    # Simple key presence check
                    condition_results.append(
                        condition in utm_dict_lower and 
                        utm_dict_lower[condition] != ''
                    )
            
            # Rule matches if all conditions are true
            rule_matched = all(condition_results)
        
        # If rule is matched, return the rule details
        if rule_matched:
            return {
                'RULE NAME': rule['RULE NAME'],
                'MARKETING_CHANNEL': rule['MARKETING_CHANNEL'],
                'MARKETING_CHANNEL_ROLLUP': rule['MARKETING_CHANNEL_ROLLUP'],
                'MARKETING_TYPE': rule['MARKETING_TYPE']
            }
    
    # No rule matched
    return None

In [ ]:
def rules_validation(input_df, rules_df):
    file = open('mkt_errors.txt','w')
    try:
        for _, row in input_df.iterrows():
            viewurl_tovalidate = row['VIEW_URL']
            referrerdomain_tovalidate = row['REFERRER_DOMAIN']
            if pd.isnull(referrerdomain_tovalidate):
                referrerdomain_tovalidate = ""
            theDict = extract_utm_params(viewurl_tovalidate, referrerdomain_tovalidate)
            #for key, value in theDict.items():
            #    print(f"Key: {key}, Value: {value}")
            active_rule = match_utm_rules(theDict, rules_df)
            #print("Input:", theDict)
            #print("Matched Rule:", active_rule)
            if (active_rule.get("MARKETING_CHANNEL") == row['MARKETING_CHANNEL']) and (active_rule.get("MARKETING_CHANNEL_ROLLUP")==row['MARKETING_CHANNEL_ROLLUP']) and (active_rule.get("MARKETING_TYPE")==row['MARKETING_TYPE']):
                count_by_rule(active_rule.get("RULE NAME"), "C")
            else:
                count_by_rule(active_rule.get("RULE NAME"), "E")
                file.write("view_url:" + viewurl_tovalidate + '\n')
                file.write("referrer_domain" + referrerdomain_tovalidate + '\n')
                file.write("    Rule based on view_url:" + active_rule.get("RULE NAME") + '\n')
                file.write("      By Rule <- vs -> Assigned" + '\n')
                file.write("     " + active_rule.get("MARKETING_CHANNEL") +        ' <----->' + row['MARKETING_CHANNEL'] + '\n')
                file.write("     " + active_rule.get("MARKETING_CHANNEL_ROLLUP") + ' <----->' + row['MARKETING_CHANNEL_ROLLUP'] + '\n')
                file.write("     " + active_rule.get("MARKETING_TYPE") +           ' <----->' + row['MARKETING_TYPE'] + '\n')
    finally:
        file.close()

In [ ]:
rules_df = create_rules_dataframe()

In [ ]:
select HTTPHEADERUSERAGENT, MPARTICLEUSERID, OTHERUSERID, OTHERUSERID3, BATCHID, CLIENTIP, COUNTRYCODE, CITYNAME, POSTALCODE, REGIONCODE, SESSIONID, SESSIONSTARTTIMESTAMP, EVENTID, EVENTDATE, EVENTHOUR, EVENTNAME, 
        MPARTICLEDEVICEID,eventtimestamp,
		EVENTATTRIBUTES:"app_name" as app_name,
		EVENTATTRIBUTES:"car_data_array" as car_data_array,
		EVENTATTRIBUTES:"car_make" as car_make,
		EVENTATTRIBUTES:"car_model" as car_model,
		EVENTATTRIBUTES:"car_segment" as car_segment,
		EVENTATTRIBUTES:"car_year" as car_year,
		EVENTATTRIBUTES:"content_brand" as content_brand,
		EVENTATTRIBUTES:"content_id" as content_id,
		EVENTATTRIBUTES:"content_modification_date" as content_modification_date,
		EVENTATTRIBUTES:"content_photographer_primary" as content_photographer_primary,
		EVENTATTRIBUTES:"content_photographer_secondary" as content_photographer_secondary,
		EVENTATTRIBUTES:"content_publication_date" as content_publication_date,
		EVENTATTRIBUTES:"content_tags" as content_tags,
		EVENTATTRIBUTES:"content_title" as content_title,
		EVENTATTRIBUTES:"content_type" as content_type,
		EVENTATTRIBUTES:"content_version" as content_version,
		EVENTATTRIBUTES:"content_writer_primary" as content_writer_primary,
		EVENTATTRIBUTES:"device_family" as device_family,
		EVENTATTRIBUTES:"device_screen_area" as device_screen_area,
		EVENTATTRIBUTES:"device_screen_height" as device_screen_height,
		EVENTATTRIBUTES:"device_screen_width" as device_screen_width,
		EVENTATTRIBUTES:"device_user_agent" as device_user_agent,
		EVENTATTRIBUTES:"inventory_widget_present" as inventory_widget_present,
		EVENTATTRIBUTES:"nitrous_version" as nitrous_version,
		EVENTATTRIBUTES:"page_id" as page_id,
		EVENTATTRIBUTES:"platform_name" as platform_name,
		EVENTATTRIBUTES:"session_id" as session_id,
		EVENTATTRIBUTES:"true_view" as true_view,
		EVENTATTRIBUTES:"view_canonical_path" as view_canonical_path,
		EVENTATTRIBUTES:"view_canonical_url" as view_canonical_url,
		EVENTATTRIBUTES:"view_count" as view_count,
		EVENTATTRIBUTES:"view_domain" as view_domain,
		EVENTATTRIBUTES:"view_group" as view_group,
		EVENTATTRIBUTES:"view_name" as view_name,
		EVENTATTRIBUTES:"view_name_previous" as view_name_previous,
		EVENTATTRIBUTES:"view_path" as view_path,
		EVENTATTRIBUTES:"view_subdomain" as view_subdomain,
		EVENTATTRIBUTES:"view_template" as view_template,
		EVENTATTRIBUTES:"view_url" as view_url,
		EVENTATTRIBUTES:"window_id" as window_id,
        USERATTRIBUTES:"firsttouch_marketing_channel" as firsttouch_marketing_channel,
        USERATTRIBUTES:"firsttouch_marketing_channel_rollup" as firsttouch_marketing_channel_rollup,
        USERATTRIBUTES:"firsttouch_marketing_type" as firsttouch_marketing_type,
        USERATTRIBUTES:"marketing_channel" as marketing_channel,
        USERATTRIBUTES:"marketing_channel_rollup" as marketing_channel_rollup,
        USERATTRIBUTES:"marketing_type" as marketing_type,
        USERATTRIBUTES:"utm_campaign" as utm_campaign,
        USERATTRIBUTES:"utm_content" as utm_content,
        USERATTRIBUTES:"utm_id" as utm_id,
        USERATTRIBUTES:"utm_medium" as utm_medium,
        USERATTRIBUTES:"utm_source" as utm_source,
        USERATTRIBUTES:"utm_term" as utm_term,
        USERATTRIBUTES:"utm_type" as utm_type,
        USERATTRIBUTES:"referrer_domin" as referrer_domain
from MP_VW_OTHEREVENTS
WHERE appenvironment = 'Production'
AND   eventdate = CURRENT_DATE - 1
AND   APPNAME = 'NextWeb'
AND   eventname = 'Page/Screen'

In [ ]:
mkt = pv_query.to_pandas()
mkt.shape

In [ ]:
mkt.columns = mkt.columns.str.upper()

In [ ]:
mkt = remove_double_quotes_from_object_columns(mkt)


In [ ]:
mkt['MARKETING_CHANNEL'] = mkt['MARKETING_CHANNEL'].fillna('')
mkt['MARKETING_CHANNEL_ROLLUP'] = mkt['MARKETING_CHANNEL_ROLLUP'].fillna('')
mkt['MARKETING_TYPE'] = mkt['MARKETING_TYPE'].fillna('')
mkt['REFERRER_DOMAIN'] = mkt['REFERRER_DOMAIN'].fillna('')
mkt['VIEW_URL'] = mkt['VIEW_URL'].fillna('')

In [ ]:
rules_validation(mkt, rules_df)

In [ ]:
print(errors_by_rule)